In [1]:
#Basic imports
import os
import numpy as np
import pandas as pd
import random
import itertools
import time
import operator
from operator import itemgetter

In [2]:
file_path = r"C:\Users\91861\Downloads\CAB_TR_RGP_Datasets 2023(1).xlsx"

#Load demand data
CAB10_flow_df= pd.read_excel(file_path, skiprows=2, nrows=10, usecols="B:K", header=None)
CAB10_flow_df.index += 1
CAB10_flow_df.columns += 1

#Load unit cost data
CAB10_cost_df = pd.read_excel(file_path, skiprows = 18, nrows=10, usecols="B:K", header=None)
CAB10_cost_df.index += 1
CAB10_cost_df.columns += 1


CAB25_flow_df= pd.read_excel(file_path, skiprows=34, nrows=25, usecols="B:Z", header=None)
CAB25_flow_df.index += 1
CAB25_flow_df.columns += 1

CAB25_cost_df = pd.read_excel(file_path, skiprows = 62, nrows=25, usecols="B:Z", header=None)
CAB25_cost_df.index += 1
CAB25_cost_df.columns += 1

TR55_flow_df= pd.read_excel(file_path, sheet_name=1, skiprows=5, nrows=55, usecols="D:BF", header=None)
TR55_flow_df.index += 1
TR55_flow_df.columns += 1

TR55_cost_df = pd.read_excel(file_path,sheet_name=1, skiprows = 63, nrows=55, usecols="D:BF", header=None)
TR55_cost_df.index += 1
TR55_cost_df.columns += 1

TR81_flow_df= pd.read_excel(file_path, sheet_name=2, skiprows=2, nrows=81, usecols="D:CF", header=None)
TR81_flow_df.index += 1
TR81_flow_df.columns += 1

TR81_cost_df = pd.read_excel(file_path,sheet_name=2, skiprows = 86, nrows=81, usecols="D:CF", header=None)
TR81_cost_df.index += 1
TR81_cost_df.columns += 1

RGP100_flow_df= pd.read_excel(file_path, sheet_name=3, skiprows=3, nrows=100, usecols="B:CW", header=None)
RGP100_flow_df.index += 1
RGP100_flow_df.columns += 1

RGP100_cost_df = pd.read_excel(file_path,sheet_name=3, skiprows =3, nrows=100, usecols="B:CW", header=None)
RGP100_cost_df.index += 1
RGP100_cost_df.columns += 1

In [3]:
#Transform csv dataframe to list
CAB10_flow = CAB10_flow_df.to_numpy()  
CAB10_cost = CAB10_cost_df.to_numpy()

CAB25_flow = CAB25_flow_df.to_numpy()  
CAB25_cost = CAB25_cost_df.to_numpy()

TR55_flow = TR55_flow_df.to_numpy()  
TR55_cost = TR55_cost_df.to_numpy()

TR81_flow = TR81_flow_df.to_numpy()  
TR81_cost = TR81_cost_df.to_numpy()

RGP100_flow = RGP100_flow_df.to_numpy()  
RGP100_cost = RGP100_cost_df.to_numpy()

In [4]:
RGP100_flow

array([[    0, 10195,  9908, ..., 10054,  9984, 10009],
       [ 9901,     0,  9970, ...,  9987, 10165, 10126],
       [ 9874,  9845,     0, ...,  9902,  9942, 10071],
       ...,
       [ 9955,  9999, 10065, ...,     0,  9954, 10108],
       [ 9858, 10076, 10108, ...,  9991,     0,  9935],
       [10176,  9962,  9912, ...,  9742,  9978,     0]], dtype=int64)

In [5]:
case1_opt_sol = 740999439
case2_opt_sol = 8374475034

In [6]:
def initial_solution(n_node, n_hub):
    node_list = list(range(1, n_node+1))
    hub_list = random.sample(node_list, n_hub)
    solution = [node if node in hub_list else random.choice(hub_list) for node in node_list]
    return solution

In [7]:
def LC_initial_solution(n_node, n_hub, cost_matrix):
    node_list = list(range(1, n_node+1))
    hub_list = random.sample(node_list, n_hub)
    solution = [0]*n_node
    
    for hub in hub_list:
        node_list.remove(hub)
        
        spoke_hub_cost = [(cost_matrix[i][hub-1], hub) for i in range(n_node) if i+1 not in hub_list]
        spoke, spoke_hub_mapping_with_lowest_cost = min(enumerate(spoke_hub_cost), key=lambda x: x[1][0])
        
        solution[hub-1] = hub
        solution[spoke] = spoke_hub_mapping_with_lowest_cost[1]
        
    return solution

In [8]:
a=LC_initial_solution(10, 3, CAB10_cost)

In [9]:
def GA_initial_population_list(n_node, n_hub, n_population,initial_pop_generator_split_prob, cost_matrix):
    initial_population_list = [
        initial_solution(n_node, n_hub)
        for _ in range(int(n_population * initial_pop_generator_split_prob))
    ] + [
        LC_initial_solution(n_node, n_hub, cost_matrix)
        for _ in range(int(n_population * (1 - initial_pop_generator_split_prob)))
    ]
    
    return initial_population_list

In [10]:
pop = GA_initial_population_list(10, 2, 10,1, CAB10_cost)      
pop

[[9, 4, 4, 4, 4, 9, 4, 4, 9, 4],
 [3, 6, 3, 3, 6, 6, 3, 3, 6, 6],
 [2, 2, 3, 3, 2, 2, 2, 2, 3, 2],
 [3, 2, 3, 3, 3, 3, 2, 3, 3, 2],
 [3, 6, 3, 3, 6, 6, 6, 6, 6, 3],
 [1, 1, 4, 4, 4, 1, 1, 4, 4, 1],
 [6, 9, 6, 9, 6, 6, 6, 9, 9, 9],
 [1, 4, 4, 4, 4, 1, 1, 1, 4, 1],
 [8, 8, 6, 8, 6, 6, 8, 8, 6, 6],
 [6, 6, 6, 5, 5, 6, 6, 5, 5, 6]]

In [11]:
def SUM_hub_flow(solution, node, flow_matrix):

    hub_spoke = {hub: [i for i, val in enumerate(solution) if val == hub and i != hub-1] for hub in set(solution)}
    hub_flows = {hub: [flow_matrix[hub-1]] + [flow_matrix[spoke_index] for spoke_index in hub_spoke[hub]] for hub in set(solution)}
    SUM_hub_flow = {hub: sum(itertools.chain.from_iterable(hub_flows[hub])) for hub in set(solution)}
    
    return SUM_hub_flow


In [12]:
SUM_hub_flow(a, 10, CAB10_flow)

{0: 434135, 10: 136885, 3: 168414, 4: 342370}

In [13]:
def alpha(flow):
    unit_flow_cost = 0
    
    if flow < 0:
        raise ValueError("Flow must be nonnegative")
    elif flow < 50000:
        unit_flow_cost = 0 + 1*flow
    elif flow < 100000:
        unit_flow_cost = 10000 + 0.8*flow
    elif flow < 200000:
        unit_flow_cost = 30000 + 0.6*flow
    else:
        unit_flow_cost = 70000 + 0.4*flow
        
    return unit_flow_cost

def cost_evaluation(array, node, flow_matrix, cost_matrix):
    SUM_cost = 0
    
    #Identify the hubs
    hubs = set(array)
    
        
   #Calculate collection and distribution cost
    for i in range(0, len(array)):
        for j in range(0, len(array)):
            SUM_cost += flow_matrix[i][j]*(cost_matrix[i][array[i]-1] + cost_matrix[array[j]-1][j])   
    
    
   #Calculate inter-hub cost
    for k in hubs:
        for m in hubs:
            if k != m:
                interhub_flow = 0
                for i in range(0, len(array)):
                    for j in range(0, len(array)):
                        if array[i] == k and array[j] == m:
                            interhub_flow += flow_matrix[i][j]                      #Calculate inter-hub flow
                
                SUM_cost +=  alpha(interhub_flow)*cost_matrix[k-1][m-1]       #Calculate inter-hub cost
    
    return SUM_cost

In [14]:
print(CAB10_cost.shape)

(10, 10)


In [15]:
example =  [4, 9, 9, 4, 4, 9, 7, 4, 9, 7]

In [16]:
cost_evaluation(example,10,CAB10_flow, CAB10_cost)

740999438.9360801

In [17]:
def GA_compute_fitness(population, node, flow_matrix, cost_matrix, alpha):
    solution_cost = []
    SUM_hub_flow_value = sum(SUM_hub_flow(population[0], node, flow_matrix).values())
    for solution in population:
        cost = cost_evaluation(solution, node, flow_matrix, cost_matrix)
        solution_cost.append((cost, solution)) # Create a list of tuples containing the cost (fitness) and solution
    return solution_cost

In [18]:
GA_compute_fitness(pop,10,CAB10_flow, CAB10_cost,0.2)

[(913165770.5420401, [9, 4, 4, 4, 4, 9, 4, 4, 9, 4]),
 (1587285818.9726, [3, 6, 3, 3, 6, 6, 3, 3, 6, 6]),
 (1531515288.6868, [2, 2, 3, 3, 2, 2, 2, 2, 3, 2]),
 (1667008477.29284, [3, 2, 3, 3, 3, 3, 2, 3, 3, 2]),
 (1498361921.7679598, [3, 6, 3, 3, 6, 6, 6, 6, 6, 3]),
 (1021648775.0396401, [1, 1, 4, 4, 4, 1, 1, 4, 4, 1]),
 (971320633.44828, [6, 9, 6, 9, 6, 6, 6, 9, 9, 9]),
 (1066499567.2980798, [1, 4, 4, 4, 4, 1, 1, 1, 4, 1]),
 (1747411678.2863994, [8, 8, 6, 8, 6, 6, 8, 8, 6, 6]),
 (1036136140.3198804, [6, 6, 6, 5, 5, 6, 6, 5, 5, 6])]

In [19]:
def GA_get_parents(GA_fitness_computation_tuple):
    sorted_chromosomes = sorted(GA_fitness_computation_tuple, key=lambda x: x[0])
    return [chromosome[1] for chromosome in sorted_chromosomes[:2]]

In [20]:
def GA_crossover_model(parents, n_node):
    random_point = random.choice(range(0, n_node))
    parent1 = random.choice(parents)
    parent2 = parents[1 - parents.index(parent1)]
    offspring = parent1[:random_point+1] + parent2[random_point+1:]
    return offspring


In [21]:
def GA_gene_reallocation(offspring, n_node, cost_matrix, unique_hub):
    spoke_list = set(range(1, n_node + 1))
    for hub in unique_hub:
        if offspring[hub - 1] != hub:
            offspring[hub - 1] = hub
    spoke_list -= set(unique_hub)
    spoke_hub_cost = {i: [(cost_matrix[i - 1][j - 1], j) for j in unique_hub] for i in spoke_list}
    spoke_hub_mapping_with_lowest_cost = {spoke: min(maps, key=lambda x: x[0])[1] for spoke, maps in spoke_hub_cost.items()}
    spoke_hub_index_mapping_with_lowest_cost = [(spoke - 1, hub - 1) for spoke, hub in spoke_hub_mapping_with_lowest_cost.items()]
    solution = offspring.copy()
    for index_mapping in spoke_hub_index_mapping_with_lowest_cost:
        solution[index_mapping[0]] = index_mapping[1] + 1
    return solution

In [22]:
def GA_repair_offspring(offspring, n_node, n_hub):
 
    # Declare input
    hubs_offspring = []
    
    # Declare unique hub in the offspring as an input for gene repair operators
    unique_hub = [hub for hub in list(set(offspring)) if hub != 0]

    # Ensure we obtain feasible solutions only
    for hub in unique_hub:
        if offspring[hub-1] != hub:
            offspring[hub-1] = hub

    # Create a dictionary from the offspring as an input for gene repair operators
    offspring_dict = dict(enumerate(offspring))

    # Gene repair operators
    if len(unique_hub) == n_hub:
        offspring_repaired = GA_gene_reallocation(offspring, n_node, cost_matrix, unique_hub) 
    elif len(unique_hub) > n_hub:
        repaired_unique_hub = random.choice(list(itertools.combinations(unique_hub, n_hub)))
        for key, value in offspring_dict.items():
            if value == 0 or value not in repaired_unique_hub:
                offspring[key] = random.choice(repaired_unique_hub)
        offspring_repaired = GA_gene_reallocation(offspring, n_node, cost_matrix, repaired_unique_hub)
    else:
        new_hub_required = n_hub - len(unique_hub)
        possible_new_hub = set(range(1,n_node+1)) - set(unique_hub)
        random_new_hub = random.choices(list(possible_new_hub), k=new_hub_required)
        repaired_unique_hub = unique_hub + random_new_hub
        for hub in random_new_hub:
            if offspring[hub-1] != hub:
                offspring[hub-1] = hub
        for key, value in offspring_dict.items():
            if value == 0 or value not in repaired_unique_hub:
                offspring[key] = random.choice(repaired_unique_hub)
        offspring_repaired = GA_gene_reallocation(offspring, n_node, cost_matrix, repaired_unique_hub)
        
    return offspring_repaired

In [23]:
def GA_mutation(offspring, n_node, mutation_rate):
    unique_hub = list(set(offspring))
    node_list = list(range(1,n_node+1))
    
    for hub in unique_hub:
        spoke_index_list = [node-1 for node in node_list if node not in unique_hub]
    mutation_counter = int(mutation_rate * n_node)
    for i in range(mutation_counter):
        node_index_to_reassign = random.choice(spoke_index_list)
        available_hubs = [hub for hub in unique_hub if hub != offspring[node_index_to_reassign]]
        mutated_offspring = offspring.copy()
        mutated_offspring[node_index_to_reassign] = random.choice(available_hubs)
    return mutated_offspring


In [24]:
def genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, SUM_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter):
    
    # Step 1: Declare inputs
    generation_count = 0
    optimal_solution_list = []
    
    # Step 2: Generate initial population
    initial_population_df = pd.DataFrame()
    counter = 2
    initial_population_counter = 0
    while initial_population_counter < counter:
        initial_population_temp = GA_initial_population_list(n_node, n_hub, n_population, initial_pop_generator_split_prob, cost_matrix)
        SUM_hub_flow_value = sum(SUM_hub_flow(initial_population_temp[0], n_node, flow_matrix).values())

        initial_population_temp_fitness = GA_compute_fitness(initial_population_temp, n_node, flow_matrix, cost_matrix, alpha)

        initial_population_temp_df = pd.DataFrame(initial_population_temp_fitness, columns=['fitness','config'])
        initial_population_counter += 1

    initial_population_df = initial_population_temp_df.sort_values('fitness',ascending=True).head(n_population)
    initial_population = initial_population_df['config'].to_list()
    
    # Step 3: Compute the fitness of each chromosome in initial population
    initial_population_fitness = [tuple(x) for x in initial_population_df.to_numpy()]
    
    old_gen = initial_population
    old_gen_fitness = initial_population_fitness
    print('Initial population is created')
    print('Initial population evaluation is done')
    
    while generation_count < SUM_generation:
        # Step 3: Select a set of individuals as parents (Tournamet selection)
        parents = GA_get_parents(old_gen_fitness) # Least cost parents
        
        crossover_random_prob = random.uniform(0, 1)
        if crossover_random_prob <= crossover_prob:
            # Step 4: Populate offspring
            counter = 0
            while counter < crossover_counter:
                offspring_crossovered = GA_crossover_model(parents, n_node)
                offspring_repaired = GA_repair_offspring(offspring_crossovered, n_node, n_hub)
                offspring_repaired_fitness = cost_evaluation(offspring_repaired, n_node, flow_matrix, cost_matrix)
                print(f'Offspring repaired fitness: {round(offspring_repaired_fitness,2)}')

                # Step 5: Offspring mutation
                offspring_mutation = GA_mutation(offspring_repaired, n_node, mutation_rate)
                offspring_mutation_fitness = cost_evaluation(offspring_mutation, n_node, flow_matrix, cost_matrix)
                print(f'Offspring mutation fitness: {round(offspring_mutation_fitness,2)}')

                # Step 6: Replace the worst fit chromosome in the last generation with the new offspring
                old_gen_worst_fitness = max(old_gen_fitness, key=itemgetter(0))[0]
                old_gen_worst_fitness_chromosome = max(old_gen_fitness, key=itemgetter(0))[1]
                print(f'Previous generation worst fitness: {round(old_gen_worst_fitness,2)}')

                # If offspring_repaired is better than old_gen and offspring_mutation, replace the worst chromosome in old_gen with offspring_repaired
                if offspring_repaired_fitness < old_gen_worst_fitness and offspring_repaired_fitness <= offspring_mutation_fitness:
                    new_gen = [offspring_repaired if old_chromosome == old_gen_worst_fitness_chromosome else old_chromosome for old_chromosome in old_gen]
                # If offspring_mutation is better than old_gen and offspring_repaired, replace the worst chromosome in old_gen with offspring_mutation
                elif offspring_mutation_fitness < old_gen_worst_fitness and offspring_mutation_fitness <= offspring_repaired_fitness:
                    new_gen = [offspring_mutation if old_chromosome == old_gen_worst_fitness_chromosome else old_chromosome for old_chromosome in old_gen]
                # Else, keep the old_gen
                else:
                    new_gen = old_gen
                    
                old_gen = new_gen
                old_gen_fitness = GA_compute_fitness(new_gen, n_node, flow_matrix, cost_matrix, alpha)
                old_gen_worst_fitness = max(old_gen_fitness, key=itemgetter(0))[0]
                print(f'Replacement {counter+1} of iteration {generation_count+1} is done')
                counter += 1
                
            # Step 7: Calculate optimal solution
            new_gen_fitness = GA_compute_fitness(new_gen, n_node, flow_matrix, cost_matrix, alpha)

            for index in list(range(0,n_population)):
                optimal_solution_cost = new_gen_fitness[index][0]
                optimal_solution_node = new_gen[index]
                optimal_solution_hub = list(set(new_gen[index]))
                optimal_solution_list.append((optimal_solution_cost, optimal_solution_node, optimal_solution_hub))

            print(f'Optimal solution of iteration {generation_count+1} has been calculated')
            print('-----------------------------------------------------------------------')
        else:
            # Step 4: Select one of the parents randomly
            random_parent = parents[0]
            random_parent_fitness = cost_evaluation(random_parent, n_node, flow_matrix, cost_matrix)
            
            # Step 5: Perform mutation if applicable
            mutation_random_prob = random.uniform(0,1)
            if mutation_random_prob <= mutation_prob:
                offspring_mutation = GA_mutation(random_parent, n_node, mutation_rate)
                offspring_mutation_fitness = cost_evaluation(offspring_mutation, n_node, flow_matrix, cost_matrix)
                print(f'Offspring mutation fitness: {round(offspring_mutation_fitness,2)}')
                
                # Step 6: Replace the worst fit chromosome in the last generation with the new offspring
                old_gen_worst_fitness = max(old_gen_fitness, key=itemgetter(0))[0]
                old_gen_worst_fitness_chromosome = max(old_gen_fitness, key=itemgetter(0))[1]
                print(f'Previous generation worst fitness: {round(old_gen_worst_fitness,2)}')
                
                if offspring_mutation_fitness < old_gen_worst_fitness and offspring_mutation_fitness <= random_parent_fitness:
                    new_gen = [offspring_mutation if old_chromosome == old_gen_worst_fitness_chromosome else old_chromosome for old_chromosome in old_gen]
                elif random_parent_fitness < old_gen_worst_fitness and random_parent_fitness <= offspring_mutation_fitness:
                    new_gen = [random_parent if old_chromosome == old_gen_worst_fitness_chromosome else old_chromosome for old_chromosome in old_gen]
                else:
                    new_gen = old_gen    
                
                # Step 7: Calculate optimal solution
                new_gen_fitness = GA_compute_fitness(new_gen, n_node, flow_matrix, cost_matrix, alpha)

                for index in list(range(0,n_population)):
                    optimal_solution_cost = new_gen_fitness[index][0]
                    optimal_solution_node = new_gen[index]
                    optimal_solution_hub = list(set(new_gen[index]))
                    optimal_solution_list.append((optimal_solution_cost, optimal_solution_node, optimal_solution_hub))

                print(f'Optimal solution of iteration {generation_count+1} has been calculated')
                print('-----------------------------------------------------------------------')
                
            else:
                offspring_random = random_parent
                offspring_random_fitness = cost_evaluation(offspring_random, n_node, flow_matrix, cost_matrix)
                print(f'Random parent fitness: {round(offspring_random_fitness,2)}')
                
                # Step 6: Replace the worst fit chromosome in the last generation with the new offspring
                old_gen_worst_fitness = max(old_gen_fitness, key=itemgetter(0))[0]
                old_gen_worst_fitness_chromosome = max(old_gen_fitness, key=itemgetter(0))[1]
                print(f'Previous generation worst fitness: {round(old_gen_worst_fitness,2)}')
                
                new_gen = [offspring_random if old_chromosome == old_gen_worst_fitness_chromosome else old_chromosome for old_chromosome in old_gen]
        
                # Step 7: Calculate optimal solution
                new_gen_fitness = GA_compute_fitness(new_gen, n_node, flow_matrix, cost_matrix, alpha)

                for index in list(range(0,n_population)):
                    optimal_solution_cost = new_gen_fitness[index][0]
                    optimal_solution_node = new_gen[index]
                    optimal_solution_hub = list(set(new_gen[index]))
                    optimal_solution_list.append((optimal_solution_cost, optimal_solution_node, optimal_solution_hub))

                print(f'Optimal solution of iteration {generation_count+1} has been calculated')
                print('-----------------------------------------------------------------------')
        
        # Step 8: Looping/Regeneration
        old_gen = new_gen
        old_gen_fitness = GA_compute_fitness(new_gen, n_node, flow_matrix, cost_matrix, alpha)
        old_gen_worst_fitness = max(old_gen_fitness, key=itemgetter(0))[0]
        generation_count += 1
            
    global_best = round(min(optimal_solution_list, key=itemgetter(0))[0],2)
    best_config = min(optimal_solution_list, key=itemgetter(0))[1]
    selected_hubs = min(optimal_solution_list, key=itemgetter(0))[2]
    
    global_best_solution = [global_best, best_config, selected_hubs]

    print(f'Global best value: {global_best}')
    print(f'Global best config: {best_config}')
    return global_best_solution

# 1) CAB  n=10, p=3 

In [25]:
%%time

start_time = time.time()


n_n_node = 10
n_hub = 3
n_population = 10
initial_pop_generator_split_prob = 0
flow_matrix = CAB10_flow
cost_matrix = CAB10_cost
SUM_generation = 10
crossover_prob = 0.9
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, SUM_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case1_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','n_node','unique_hub'])
avg_tnc = round(case1_df["cost"].mean(),2)
best_tnc = round(case1_df["cost"].min(),2)
gap = round((best_tnc - case1_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case1_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring repaired fitness: 793330741.32
Offspring mutation fitness: 800393731.82
Previous generation worst fitness: 1977180217.19
Replacement 1 of iteration 1 is done
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 897598227.79
Offspring mutation fitness: 924266720.76
Previous generation worst fitness: 1964956061.83
Replacement 1 of iteration 2 is done
Optimal solution of iteration 2 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 897598227.79
Offspring mutation fitness: 930823045.74
Previous generation worst fitness: 1662248338.68
Replacement 1 of iteration 3 is done
Optimal solution of iteration 3 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 897598227.79
Offspring mutat

# 2) CAB n=25, p=3

In [26]:
%%time

start_time = time.time()


n_node = 25
n_hub = 3
n_population = 25
initial_pop_generator_split_prob = 0
flow_matrix = CAB25_flow
cost_matrix = CAB25_cost
SUM_generation = 10
crossover_prob = 0.9
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, SUM_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case2_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case2_df["cost"].mean(),2)
best_tnc = round(case2_df["cost"].min(),2)
gap = round((best_tnc - case2_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case2_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring repaired fitness: 12945993729.84
Offspring mutation fitness: 12963026781.66
Previous generation worst fitness: 21787030103.44
Replacement 1 of iteration 1 is done
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Offspring mutation fitness: 19688295045.19
Previous generation worst fitness: 14859671651.05
Optimal solution of iteration 2 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 11588650141.99
Offspring mutation fitness: 13909101609.86
Previous generation worst fitness: 14789502106.8
Replacement 1 of iteration 3 is done
Optimal solution of iteration 3 has been calculated
-----------------------------------------------------------------------
Random parent fitness: 10694365244.41
Previous generation worst fitness: 14694727613.97
Optimal solution of iteration 4 ha

# 3) CAB n=25, p=5


In [27]:
%%time

start_time = time.time()


n_node = 25
n_hub = 5
n_population = 25
initial_pop_generator_split_prob = 0
flow_matrix = CAB25_flow
cost_matrix = CAB25_cost
SUM_generation = 10
crossover_prob = 0.9
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, SUM_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case2_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case2_df["cost"].mean(),2)
best_tnc = round(case2_df["cost"].min(),2)
gap = round((best_tnc - case2_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'Optimal solution answer: {case2_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring repaired fitness: 9071911249.4
Offspring mutation fitness: 9258312710.51
Previous generation worst fitness: 20878967199.57
Replacement 1 of iteration 1 is done
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 9071911249.4
Offspring mutation fitness: 9342044661.79
Previous generation worst fitness: 18515903903.84
Replacement 1 of iteration 2 is done
Optimal solution of iteration 2 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 9071911249.4
Offspring mutation fitness: 9268813342.6
Previous generation worst fitness: 16708856478.74
Replacement 1 of iteration 3 is done
Optimal solution of iteration 3 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 9071911249.4
Offspring 

# 4) TR n=55, p=3

In [28]:
%%time

start_time = time.time()


n_node = 55
n_hub = 3
n_population = 25
initial_pop_generator_split_prob = 0
flow_matrix = TR55_flow
cost_matrix = TR55_cost
SUM_generation = 10
crossover_prob = 0.9
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, SUM_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case3_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case3_df["cost"].mean(),2)
best_tnc = round(case3_df["cost"].min(),2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')


Initial population is created
Initial population evaluation is done
Offspring repaired fitness: 45386043373.2
Offspring mutation fitness: 47676174786.2
Previous generation worst fitness: 57642276558.4
Replacement 1 of iteration 1 is done
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 40087563685.6
Offspring mutation fitness: 40327956394.0
Previous generation worst fitness: 55388690993.2
Replacement 1 of iteration 2 is done
Optimal solution of iteration 2 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 45386043373.2
Offspring mutation fitness: 46373667139.6
Previous generation worst fitness: 55160801705.8
Replacement 1 of iteration 3 is done
Optimal solution of iteration 3 has been calculated
-----------------------------------------------------------------------
Random parent fitness: 40087563685.6
Previous gene

# 5) TR n=55, p=5

In [29]:
%%time

start_time = time.time()


n_node = 55
n_hub = 5
n_population = 25
initial_pop_generator_split_prob = 0
flow_matrix = TR55_flow
cost_matrix = TR55_cost
SUM_generation = 10
crossover_prob = 0.9
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, SUM_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case3_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case3_df["cost"].mean(),2)
best_tnc = round(case3_df["cost"].min(),2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')

Initial population is created
Initial population evaluation is done
Offspring repaired fitness: 26879548642.2
Offspring mutation fitness: 26932124697.8
Previous generation worst fitness: 60572375072.0
Replacement 1 of iteration 1 is done
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 26879548642.2
Offspring mutation fitness: 29208037075.0
Previous generation worst fitness: 58552402613.4
Replacement 1 of iteration 2 is done
Optimal solution of iteration 2 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 26879548642.2
Offspring mutation fitness: 27266045240.2
Previous generation worst fitness: 57822014164.4
Replacement 1 of iteration 3 is done
Optimal solution of iteration 3 has been calculated
-----------------------------------------------------------------------
Random parent fitness: 26879548642.2
Previous gene

# 6) TR81 n=81, p=5


In [30]:
%%time

start_time = time.time()


n_node = 81
n_hub = 5
n_population = 25
initial_pop_generator_split_prob = 0
flow_matrix = TR81_flow
cost_matrix = TR81_cost
SUM_generation = 10
crossover_prob = 0.9
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, SUM_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case4_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case4_df["cost"].mean(),2)
best_tnc = round(case4_df["cost"].min(),2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')

Initial population is created
Initial population evaluation is done
Offspring repaired fitness: 54518907899.79
Offspring mutation fitness: 54619057316.72
Previous generation worst fitness: 109830830243.73
Replacement 1 of iteration 1 is done
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 50105281159.08
Offspring mutation fitness: 50313344091.01
Previous generation worst fitness: 109229692121.4
Replacement 1 of iteration 2 is done
Optimal solution of iteration 2 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 54284969356.41
Offspring mutation fitness: 54780528771.96
Previous generation worst fitness: 80146383698.93
Replacement 1 of iteration 3 is done
Optimal solution of iteration 3 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 49873661565.

# 7) TR81 n=81, p=7

In [31]:
%%time

start_time = time.time()


n_node = 81
n_hub = 7
n_population = 25
initial_pop_generator_split_prob = 0
flow_matrix = TR81_flow
cost_matrix = TR81_cost
SUM_generation = 10
crossover_prob = 0.9
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, SUM_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case4_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case4_df["cost"].mean(),2)
best_tnc = round(case4_df["cost"].min(),2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')

Initial population is created
Initial population evaluation is done
Offspring repaired fitness: 70995970603.24
Offspring mutation fitness: 72082179599.04
Previous generation worst fitness: 82923061048.43
Replacement 1 of iteration 1 is done
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 70995970603.24
Offspring mutation fitness: 72432782553.59
Previous generation worst fitness: 82363969942.83
Replacement 1 of iteration 2 is done
Optimal solution of iteration 2 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 70995970603.24
Offspring mutation fitness: 78203692909.4
Previous generation worst fitness: 80710211407.25
Replacement 1 of iteration 3 is done
Optimal solution of iteration 3 has been calculated
-----------------------------------------------------------------------
Random parent fitness: 70995970603.24
Prev

# 8) RGP100 n=100, p=7

In [32]:
%%time

start_time = time.time()


n_node = 100
n_hub = 7
n_population = 25
initial_pop_generator_split_prob = 0
flow_matrix = RGP100_flow
cost_matrix = RGP100_cost
SUM_generation = 10
crossover_prob = 0.9
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, SUM_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case5_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case5_df["cost"].mean(),2)
best_tnc = round(case5_df["cost"].min(),2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')

Initial population is created
Initial population evaluation is done
Offspring repaired fitness: 2198711403859.8
Offspring mutation fitness: 2198957651914.2
Previous generation worst fitness: 1968454243748.2
Replacement 1 of iteration 1 is done
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 2198233045706.2
Offspring mutation fitness: 2197927019414.2
Previous generation worst fitness: 1968454243748.2
Replacement 1 of iteration 2 is done
Optimal solution of iteration 2 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 2198009638012.2
Offspring mutation fitness: 2197483765037.8
Previous generation worst fitness: 1968454243748.2
Replacement 1 of iteration 3 is done
Optimal solution of iteration 3 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 2198

# 9) RGB n=100, p=10

In [33]:
%%time

start_time = time.time()


n_node = 100
n_hub = 10
n_population = 25
initial_pop_generator_split_prob = 0
flow_matrix = RGP100_flow
cost_matrix = RGP100_cost
SUM_generation = 10
crossover_prob = 0.9
mutation_prob = 0.1
mutation_rate = 0.1
crossover_counter = 1

global_minimum_solution_list = []
replication = 0
n_replication = 10

while replication < n_replication:
    GA_result = genetic_algorithm(n_node, n_hub, n_population, initial_pop_generator_split_prob, flow_matrix, cost_matrix, SUM_generation, crossover_prob, mutation_prob, mutation_rate, crossover_counter)
    global_minimum_solution_list.append(GA_result)
    print(f'====================== REPLICATION {replication} IS FINISHED ======================\n')
    print('\n')
    replication += 1
    
case9_df = pd.DataFrame(global_minimum_solution_list, columns=['cost','node','unique_hub'])
avg_tnc = round(case9_df["cost"].mean(),2)
best_tnc = round(case9_df["cost"].min(),2)
#gap = round((best_tnc - case5_opt_sol)/best_tnc*100,2)
avg_cpu_time = round((time.time() - start_time)/n_replication,2)

print(f'AVG CPU time: {avg_cpu_time} seconds')
#print(f'Optimal solution answer: {case5_opt_sol}')
print(f'My optimal solution: {best_tnc}')
print(f'AVG optimal solution: {avg_tnc}')
#print(f'Gap (%): {gap}%')

Initial population is created
Initial population evaluation is done
Offspring repaired fitness: 2187452841726.4
Offspring mutation fitness: 2187573318168.6
Previous generation worst fitness: 1954219835008.2
Replacement 1 of iteration 1 is done
Optimal solution of iteration 1 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 2188748631459.6
Offspring mutation fitness: 2188863209570.4
Previous generation worst fitness: 1954219835008.2
Replacement 1 of iteration 2 is done
Optimal solution of iteration 2 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 2190213692389.6
Offspring mutation fitness: 2190766644677.0
Previous generation worst fitness: 1954219835008.2
Replacement 1 of iteration 3 is done
Optimal solution of iteration 3 has been calculated
-----------------------------------------------------------------------
Offspring repaired fitness: 2188